In [ ]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] =
# Initialize Hugging Face transformer model
model_name = "gpt2"  # You can replace with other smaller open-source models from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Initialize LangChain with Hugging Face, providing the model_name as repo_id
# llm = HuggingFaceHub(model=model, tokenizer=tokenizer)  # This line is replaced
llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.7, "max_new_tokens":512}) # provide repo_id instead of the model and tokenizer

# Define the Prompt Template
prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="""
    Create an outline for a PowerPoint presentation on the topic '{topic}'.
    The outline should include the title and key points for each slide.
    Aim for 5-6 slides with concise, relevant information.
    """
)

# Create an LLMChain for slide generation
chain = LLMChain(llm=llm, prompt=prompt_template)

# Function to generate slide content using the LLM
def generate_slide_content(topic):
    # Run the chain to get a response
    response = chain.run({"topic": topic})
    slides_content = response.split("\n\n")  # Assume each slide's content is separated by double newlines
    return slides_content

# Function to create a PowerPoint presentation
def create_ppt(slide_contents, output_filename="AI_Presentation.pptx"):
    # Initialize a PowerPoint Presentation
    prs = Presentation()

    # Iterate over slide contents
    for content in slide_contents:
        slide = prs.slides.add_slide(prs.slide_layouts[1])  # Use a layout with title and content

        # Split content by title and body
        lines = content.split("\n")
        title = lines[0] if lines else "Title"
        body = "\n".join(lines[1:])

        # Set slide title
        slide.shapes.title.text = title

        # Add slide content
        text_box = slide.shapes.add_textbox(Inches(1), Inches(1.5), Inches(8), Inches(4))
        tf = text_box.text_frame
        p = tf.add_paragraph()
        p.text = body
        p.font.size = Pt(18)
        p.alignment = PP_ALIGN.LEFT

    # Save the PowerPoint presentation
    prs.save(output_filename)
    print(f"Presentation saved as {output_filename}")

# Usage example
if __name__ == "__main__":
    topic = "Machine Learning Basics"  # Example topic
    slide_contents = generate_slide_content(topic)  # Get slide content from LLM
    create_ppt(slide_contents, "Machine_Learning_Presentation.pptx")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-10-e8765639d988>:31: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
<ipython-input-10-e8765639d988>:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"topic": topic})


Presentation saved as Machine_Learning_Presentation.pptx
